# Extração de metadados de arquivos de imagens  

In [1]:
from PIL import Image, JpegImagePlugin
from PIL.ExifTags import TAGS
from pillow_heif import register_heif_opener
import piexif
import pyproj
from pathlib import Path
import os
from simplekml import Kml, Style, Color
import requests

register_heif_opener()

## Função `get_image_aspect_ratio(caminho | Image)`
> Identifica a relação de aspecto da imagem

In [2]:
def get_image_aspect_ratio(img_path):   
    image = Image.open(img_path) if type(img_path) == str else img_path
    image_aspect_ratio = image.width/image.height
    
    return image_aspect_ratio

## Função `copy_image(caminho_de_origem, caminho_destino, format_da_imagem='jpeg', extensão_do_arquivo='jpg')`  
> Copia a imagem para a pasta de destino e converte para o formato indicado

In [3]:
def copy_image(img_path, target_path, image_format='JPEG', image_extension='jpg'):
    output_image_filename = f'{Path(img_path).stem}.{image_extension}'
    target_file = f'{target_path}/{output_image_filename}'
    image = Image.open(img_path)
    image.save(target_file, image_format, exif=image.info.get('exif'))

## Função `converter(coordenadas, sinal= 1 | -1)` 
> Converte as coordenadas extraídas dos metadados da foto para o formato graus decimais 

In [4]:
def converter(coord, signal=1):
    graus = coord[0][0]/coord[0][1]
    minutos = coord[1][0]/coord[1][1]
    segundos = coord[2][0]/coord[2][1]
    return (graus + minutos/60 + segundos/3600) * signal

## Função `exif_to_tag(exif_dict)`  
> Converte o dicionário exif em um dicionário com tags exif

In [5]:
def exif_to_tag(exif_dict):
    codec = 'ISO-8859-1'
    exif_tag_dict = {}
    thumbnail = exif_dict.pop('thumbnail')
    exif_tag_dict['thumbnail'] = thumbnail.decode(codec) if thumbnail else None

    for ifd in exif_dict:
        exif_tag_dict[ifd] = {}
        for tag in exif_dict[ifd]:
            try:
                element = exif_dict[ifd][tag].decode(codec)

            except AttributeError:
                element = exif_dict[ifd][tag]

            exif_tag_dict[ifd][piexif.TAGS[ifd][tag]["name"]] = element

    return exif_tag_dict

## Função `extract_metadata(img_path | Image)`  
> Extrai metadados da foto

In [6]:
def extract_metadata(img_path):
    image = Image.open(img_path) if type(img_path) == str else img_path
    exif_dict = piexif.load(image.info.get('exif'))
    return exif_to_tag(exif_dict)

## Função `extract_gps(exif_tag_dict)`  
> Extrai informações de GPS registradas nos metadados da foto

In [7]:
def extract_gps(exif_tag_dict):
    #gps = exif_tag_dict['GPS']
    coordinates = None
    gps = exif_tag_dict.get('GPS')
    if gps:
        latitude = converter(gps['GPSLatitude'], 1 if gps['GPSLatitudeRef'].upper() == 'N' else -1)
        longitude = converter(gps['GPSLongitude'], -1 if gps['GPSLongitudeRef'].upper() == 'W' else 1)
        altitude = gps['GPSAltitude'][0]/gps['GPSAltitude'][1]
        coordinates = (round(longitude, 6), round(latitude, 6), round(altitude, 3))
    
    return coordinates

## Função `extract_datatime(exif_tag_dict)`  
> Extrai informações de data e hora registradas nos metadados da foto

In [8]:
def extract_datatime(exif_tag_dict):
    return exif_tag_dict['Exif']['DateTimeOriginal']

## Função `utm(coordinates)`  
> Converte coordenadas geográficas em coordenadas UTM. Nesse caso a altitude é ignorada.

In [9]:
def utm(coordinates):

    def zone(coordinates):
        if 56 <= coordinates[1] < 64 and 3 <= coordinates[0] < 12:
            return 32
        if 72 <= coordinates[1] < 84 and 0 <= coordinates[0] < 42:
            if coordinates[0] < 9:
                return 31
            elif coordinates[0] < 21:
                return 33
            elif coordinates[0] < 33:
                return 35
            return 37
        return int((coordinates[0] + 180) / 6) + 1


    def letter(coordinates):
        return 'CDEFGHJKLMNPQRSTUVWXX'[int((coordinates[1] + 80) / 8)]

    utm_coordinates = None
    if coordinates:
        _projections = {}

        z = zone(coordinates)
        l = letter(coordinates)
        if z not in _projections:
            _projections[z] = pyproj.Proj(proj='utm', zone=z, ellps='aust_SA')

        x, y = _projections[z](coordinates[0], coordinates[1])
        if y < 0:
            y += 10000000
        utm_coordinates = (z, l, round(x, 2), round(y, 2))
    
    return utm_coordinates

## Função `get_files(base_path, recursive=False)`  
> Coleta os arquivos de image presentes nos diretórios indicados em base_path (str ou list). Também faz busca recursiva, se `recursive=True`  

In [10]:
def get_files(base_path, recursive=False):
    files = []
    if type(base_path) is list:
        for bp in base_path:
            files += get_files(bp, recursive=recursive)
    else:
        for filename in os.listdir(base_path):
            path = os.path.join(base_path, filename)
            if os.path.isfile(path):
                if Path(path).suffix in ['.jpg', '.jpeg', '.heic']:
                    files += [path]
            elif recursive:
                files += get_files(path, recursive=recursive)
            
    return files

## Função `fetch_data(image_base_paths, recursive=False, verbose=False)`  
> Extrai metadados dados dos arquivo das images indicadas e retorna os dados em formato dicionário com o nome do arquivo (sem extensão) como chave (point_name).

In [11]:
def fetch_data(image_base_paths, recursive=False, verbose=False):
    data = {}
    paths = get_files(image_base_paths, recursive=recursive)
    for p, file in enumerate(paths):
        
        if verbose:
            print('Coletando dados ........... ', f'{round(p/len(paths)*100, 2)}%', end='\r')
        
        point_name = Path(file).stem
        metadata = extract_metadata(file)
        geo_coord = extract_gps(metadata)
        date_time = extract_datatime(metadata)
        utm_coord = utm(geo_coord)
        image_aspect_ratio = get_image_aspect_ratio(file)
        
        item = {}        
        item['file_path'] = file
        item['longitude'] = geo_coord[0] if geo_coord else None
        item['latitude'] = geo_coord[1] if geo_coord else None
        item['altitude'] = geo_coord[2] if geo_coord else None
        item['utm_zone'] = utm_coord[0] if utm_coord else None
        item['utm_letter'] = utm_coord[1] if utm_coord else None
        item['utm_x'] = utm_coord[2] if utm_coord else None
        item['utm_y'] = utm_coord[3] if utm_coord else None
        item['image_aspect_ratio'] = image_aspect_ratio
        item['date_time'] = date_time
        
        data[point_name] = item
        
    if verbose:
        print('Coletando dados ........... ', 'Concluído!')
    
    return data


## Função `create_kml(kml_filename, image_base_paths, recursive=False, output_folder='.', images_folder_name = 'images', verbose=False)`  
> Cria um arquivo kml com base nas images indicadas em image_base_paths e grava o arquivo em output_folder junto com o diretório images_folder_name, no qual as imagens convertidas para o formato padrão são copiadas.

In [12]:
import logging

def create_kml(kml_filename, image_base_paths, recursive=False, output_folder='.', 
               images_folder_name = 'images', verbose=False):

    logging.basicConfig(filename=f'{output_folder}/create_kml.log', filemode='w', 
                    format='%(asctime)s %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
    
    kml_output_path = f'{output_folder}/{kml_filename}'
    output_images_folder = f'{output_folder}/{images_folder_name}'
 
    kml = Kml()
    style = Style()
    exception = False
    
    try:
        
        logging.info('Gerando kml ...')
        
        Path(output_images_folder).mkdir(parents=True, exist_ok=True)
        if any(Path(output_images_folder).iterdir()):
            raise Exception(f'Diretório "{output_images_folder}" deve estar vazio!')

        icon_href = 'http://maps.google.com/mapfiles/kml/paddle/purple-circle.png'
        if requests.get(icon_href).status_code == 200:
            style.iconstyle.icon.href = icon_href

        metadata = fetch_data(image_base_paths=image_base_paths, 
                              recursive=recursive, verbose=verbose)

        for p, point_name in enumerate(metadata):

            file_path = metadata[point_name]['file_path']
            if file_path.startswith(f'{output_images_folder}/'):                
                raise Exception('Arquivo de origem não pode estar no diretório destino!')

            item = metadata.get(point_name)
            geo_coord = (item.get('longitude'), item.get('latitude'), item.get('altitude'))
            if not any(coord is None for coord in geo_coord):
                output_image_filename = f'{Path(file_path).stem}.jpg'
                image_aspect_ratio = item.get('image_aspect_ratio')
                date_time = item.get('date_time')

                copy_image(file_path, output_images_folder)

                point = kml.newpoint(name=f'{point_name}', coords=[geo_coord])
                point.style = style
                if image_aspect_ratio > 1:
                    image_width = 1200
                    image_height = round(image_width/image_aspect_ratio)
                else:
                    image_height = 1200
                    image_width = round(image_height*image_aspect_ratio)

                point.description = f'''<h2 style="background-color: #7700bb; color: #ffffff">
                                            Data/Hora: {date_time}
                                        </h2>
                                        <img 
                                          src="{images_folder_name}/{output_image_filename}" 
                                          alt="picture" 
                                          width="{image_width}" 
                                          height="{image_height}" 
                                          align="left" />;'''

                logging.info(f'{point_name}: ok')

            else:
                logging.critical(f'{point_name}: imagem sem informações de GPS')

            if verbose:
                print('Gerando kml ........... ', 
                      f'{round((p+1)/len(metadata.keys())*100, 2)}%', end='\r')
    
        print('\n')
        
    except Exception as e:
        exception = True
        if verbose:
            print(str(e))
            
        logging.critical(str(e))
    
    kml.save(kml_output_path)

    msg = ''
    if kml.allgeometries:
        if len(kml.allgeometries) < len(metadata.keys()):
            msg = f'Apenas {len(kml.allgeometries)} de {len(metadata.keys())} \
                    imagens com informações de GPS.'
            
            logging.warning(msg)
        else:
            msg = 'Informações de GPS coletadas de todas as imagens.'
            logging.info(msg)
    elif not exception:
        msg = 'Nenhuma imagem com informações de GPS!'
        logging.warning(msg)

    if verbose:
        print(msg)


## Teste de execução da função `fetch_data()`  

In [13]:
fetch_data(image_base_paths=['./tests/samples'], recursive=True, verbose=True)

Coletando dados ...........  Concluído!


{'IMG_20210924_113255296': {'file_path': './tests/samples/IMG_20210924_113255296.jpg',
  'longitude': -51.735524,
  'latitude': -0.183027,
  'altitude': 12.253,
  'utm_zone': 22,
  'utm_letter': 'M',
  'utm_x': 418152.45,
  'utm_y': 9979768.27,
  'image_aspect_ratio': 1.3333333333333333,
  'date_time': '2021:09:24 11:32:55'},
 'IMG_20210916_140829767': {'file_path': './tests/samples/IMG_20210916_140829767.jpg',
  'longitude': -50.9737,
  'latitude': 2.486679,
  'altitude': 21.602,
  'utm_zone': 22,
  'utm_letter': 'N',
  'utm_x': 502923.8,
  'utm_y': 274855.45,
  'image_aspect_ratio': 1.3333333333333333,
  'date_time': '2021:09:16 14:08:29'},
 'IMG_20210924_104917945': {'file_path': './tests/samples/IMG_20210924_104917945.jpg',
  'longitude': -51.728321,
  'latitude': -0.085532,
  'altitude': 15.909,
  'utm_zone': 22,
  'utm_letter': 'M',
  'utm_x': 418953.71,
  'utm_y': 9990545.34,
  'image_aspect_ratio': 1.3333333333333333,
  'date_time': '2021:09:24 10:49:17'},
 'IMG_20210916_112512

## Teste de execução da função `create_kml()`  

In [14]:
create_kml(kml_filename='locais.kml', output_folder='./tests', image_base_paths=['./tests/samples'], recursive=True, verbose=True)

Coletando dados ...........  Concluído!
Gerando kml ...........  100.0%

Informações de GPS coletadas de todas as imagens.
